## Mixed precision training

This module allows the forward and backward passes of your neural net to be done in fp16 (also known as *half precision*). This is particularly important if you have an NVIDIA GPU with [tensor cores](https://www.nvidia.com/en-us/data-center/tensorcore/), since it can speed up your training by 200% or more.

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai.callbacks.fp16 import *
from fastai.vision import *

## Overview

To train your model in mixed precision you just have to call [`Learner.to_fp16`](/train.html#to_fp16), which converts the model and modifies the existing [`Learner`](/basic_train.html#Learner) to add [`MixedPrecision`](/callbacks.fp16.html#MixedPrecision).

In [ ]:
show_doc(Learner.to_fp16)

<h4 id="to_fp16" class="doc_header"><code>to_fp16</code><a href="https://github.com/fastai/fastai/blob/master/fastai/train.py#L43" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#to_fp16-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>to_fp16</code>(**`learn`**:[`Learner`](/basic_train.html#Learner), **`loss_scale`**:`float`=***`None`***, **`max_noskip`**:`int`=***`1000`***, **`dynamic`**:`bool`=***`True`***, **`clip`**:`float`=***`None`***, **`flat_master`**:`bool`=***`False`***, **`max_scale`**:`float`=***`16777216`***, **`loss_fp32`**:`bool`=***`True`***) → [`Learner`](/basic_train.html#Learner)

<div class="collapse" id="to_fp16-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#to_fp16-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>to_fp16</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Put `learn` in FP16 precision mode.  

For example:

In [ ]:
path = untar_data(URLs.MNIST_SAMPLE)
data = ImageDataBunch.from_folder(path)
model = simple_cnn((3,16,16,2))
learn = Learner(data, model, metrics=[accuracy]).to_fp16()
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.126117,0.117945,0.956820,00:03


In [ ]:
jekyll_warn("Known issue: `Learner.to_fp16` doesn't work when training GANs.")

<div markdown="span" class="alert alert-danger" role="alert"><i class="fa fa-danger-circle"></i> <b>Warning: </b>Known issue: `Learner.to_fp16` doesn't work when training GANs.</div>

Details about mixed precision training are available in [NVIDIA's documentation](https://docs.nvidia.com/deeplearning/sdk/mixed-precision-training/index.html). We will just summarize the basics here.

The only parameter you may want to tweak is `loss_scale`. This is used to scale the loss up, so that it doesn't underflow fp16, leading to loss of accuracy (this is reversed for the final gradient calculation after converting back to fp32). Generally the default `512` works well, however. You can also enable or disable the flattening of the master parameter tensor with `flat_master=True`, however in our testing the different is negligible.

Internally, the callback ensures that all model parameters (except batchnorm layers, which require fp32) are converted to fp16, and an fp32 copy is also saved. The fp32 copy (the `master` parameters) is what is used for actually updating with the optimizer; the fp16 parameters are used for calculating gradients. This helps avoid underflow with small learning rates.

All of this is implemented by the following Callback.

In [ ]:
show_doc(MixedPrecision)

<h2 id="MixedPrecision" class="doc_header"><code>class</code> <code>MixedPrecision</code><a href="https://github.com/fastai/fastai/blob/master/fastai/callbacks/fp16.py#L64" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#MixedPrecision-pytest" style="float:right; padding-right:10px">[test]</a></h2>

> <code>MixedPrecision</code>(**`learn`**:[`Learner`](/basic_train.html#Learner), **`loss_scale`**:`float`=***`None`***, **`max_noskip`**:`int`=***`1000`***, **`dynamic`**:`bool`=***`True`***, **`clip`**:`float`=***`None`***, **`flat_master`**:`bool`=***`False`***, **`max_scale`**:`float`=***`16777216`***, **`loss_fp32`**:`bool`=***`True`***) :: [`LearnerCallback`](/basic_train.html#LearnerCallback)

<div class="collapse" id="MixedPrecision-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#MixedPrecision-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>MixedPrecision</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Base class for creating callbacks for a [`Learner`](/basic_train.html#Learner).  

### Callback methods

You don't have to call the following functions yourself - they're called by fastai's [`Callback`](/callback.html#Callback) system automatically to enable the class's functionality.

In [ ]:
show_doc(MixedPrecision.on_backward_begin)

<h4 id="MixedPrecision.on_backward_begin" class="doc_header"><code>on_backward_begin</code><a href="https://github.com/fastai/fastai/blob/master/fastai/callbacks/fp16.py#L93" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#MixedPrecision-on_backward_begin-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>on_backward_begin</code>(**`last_loss`**:`Rank0Tensor`, **\*\*`kwargs`**:`Any`) → `Rank0Tensor`

<div class="collapse" id="MixedPrecision-on_backward_begin-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#MixedPrecision-on_backward_begin-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>on_backward_begin</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Scale gradients up by `self.loss_scale` to prevent underflow.  

In [ ]:
show_doc(MixedPrecision.on_backward_end)

<h4 id="MixedPrecision.on_backward_end" class="doc_header"><code>on_backward_end</code><a href="https://github.com/fastai/fastai/blob/master/fastai/callbacks/fp16.py#L99" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#MixedPrecision-on_backward_end-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>on_backward_end</code>(**\*\*`kwargs`**:`Any`)

<div class="collapse" id="MixedPrecision-on_backward_end-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#MixedPrecision-on_backward_end-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>on_backward_end</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Convert the gradients back to FP32 and divide them by the scale.  

In [ ]:
show_doc(MixedPrecision.on_loss_begin)

<h4 id="MixedPrecision.on_loss_begin" class="doc_header"><code>on_loss_begin</code><a href="https://github.com/fastai/fastai/blob/master/fastai/callbacks/fp16.py#L89" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#MixedPrecision-on_loss_begin-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>on_loss_begin</code>(**`last_output`**:`Tensor`, **\*\*`kwargs`**:`Any`) → `Tensor`

<div class="collapse" id="MixedPrecision-on_loss_begin-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#MixedPrecision-on_loss_begin-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>on_loss_begin</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Convert half precision output to FP32 to avoid reduction overflow.  

In [ ]:
show_doc(MixedPrecision.on_step_end)

<h4 id="MixedPrecision.on_step_end" class="doc_header"><code>on_step_end</code><a href="https://github.com/fastai/fastai/blob/master/fastai/callbacks/fp16.py#L118" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#MixedPrecision-on_step_end-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>on_step_end</code>(**\*\*`kwargs`**:`Any`)

<div class="collapse" id="MixedPrecision-on_step_end-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#MixedPrecision-on_step_end-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>on_step_end</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Update the params from master to model and zero grad.  

In [ ]:
show_doc(MixedPrecision.on_train_begin)

<h4 id="MixedPrecision.on_train_begin" class="doc_header"><code>on_train_begin</code><a href="https://github.com/fastai/fastai/blob/master/fastai/callbacks/fp16.py#L77" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#MixedPrecision-on_train_begin-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>on_train_begin</code>(**\*\*`kwargs`**:`Any`)

<div class="collapse" id="MixedPrecision-on_train_begin-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#MixedPrecision-on_train_begin-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>on_train_begin</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Prepare the master model.  